In [1]:
import sys

sys.path.append("../")

In [2]:
import torch
import torch.sparse

In [5]:
idxs = [[0, 0],
        [0, 2]]
values = [1, 1]

In [6]:
st = torch.sparse_coo_tensor(idxs, values, size=(1,10), dtype=torch.float32)
st.to_dense()

tensor([[1., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [7]:
t = torch.arange(10).reshape(10, 1).float()+1

In [8]:
torch.matmul(st, t)

tensor([[4.]])

## Testing linear layer

In [9]:
torch.sparse_coo_tensor(
    [[0, 1, 2, 3],
     [0, 0, 2, 2]],
    [1, 1, 1, 1],
    size=(4, 3)
).to_dense()

tensor([[1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1]])

In [23]:
torch.sparse_coo_tensor(
    [[0, 0, 1, 2, 3],
     [0, 2, 0, 2, 2]],
    [1, 1, 1, 1, 1],
    size=(4, 3)
)[0].to_dense()

tensor([1, 0, 1])

In [1]:
import sys
sys.path.append("../")


import torch
import torch.sparse
import torch.nn as nn
from torch.utils.data import Dataset
import pandas as pd
from torch.utils.data import DataLoader

from utils.dataset import RatingsDatasetGenre

In [2]:
genres_map = {
    "Action": 0,
    "Adventure": 1,
    "Animation": 2,
    "Children's": 3,
    "Comedy": 4,
    "Crime": 5,
    "Documentary": 6,
    "Drama": 7,
    "Fantasy": 8,
    "Film-Noir": 9,
    "Horror": 10,
    "Musical": 11,
    "Mystery": 12,
    "Romance": 13,
    "Sci-Fi": 14,
    "Thriller": 15,
    "War": 16,
    "Western": 17,
}

movies_csv = pd.read_csv("../train_data/movies_data.csv")
movies_csv["movie_id"] = movies_csv["movie_id"].apply(lambda x: x - 1)
movies_csv.head()

,movie_id,title,genres
0,0,Toy Story (1995),Animation|Children's|Comedy
1,1,Jumanji (1995),Adventure|Children's|Fantasy
2,2,Grumpier Old Men (1995),Comedy|Romance
3,3,Waiting to Exhale (1995),Comedy|Drama
4,4,Father of the Bride Part II (1995),Comedy


In [3]:
users = 3974
movies = 3564

train_dataloader = DataLoader(
    RatingsDatasetGenre(
        users,
        movies,
        "../train_data/train.csv",
        "../train_data/movies_data.csv",
        "user_id",
        "movie_id",
        "rating",
        "genres",
        genres_map,
    ), 
    batch_size=4,
    num_workers=1,
    shuffle=True,
)

In [6]:
for i_batch, (vus, vis, rs, gs) in enumerate(train_dataloader):    
    arange = torch.arange(vus.shape[0])
    ones = torch.ones(vus.shape[0], dtype=torch.float64)
    
    sp_vus = torch.sparse_coo_tensor(
        torch.vstack([arange, vus]),
        ones,
        size=(vus.shape[0], users),
        dtype=torch.float32,
    )
    sp_vis = torch.sparse_coo_tensor(
        torch.vstack([arange, vis]),
        ones,
        size=(vis.shape[0], movies),
        dtype=torch.float32,
    )
    
    g_vis = torch.cat([sp_vis, gs.to_sparse()], dim=1)
    
    layer = nn.Linear(movies+18, 2)
    out = layer(g_vis)
    
    print(out)
    
    break

cpu
tensor([[ 0.0023, -0.0102],
        [-0.0198, -0.0242],
        [-0.0144, -0.0108],
        [-0.0088, -0.0235]], grad_fn=<AddmmBackward0>)


## Sparse data loader